In [1]:
from query_decomposer import QueryDecomposer

In [2]:
qd = QueryDecomposer()

Loading transformer model and tokenizer from transformers library: meta-llama/Meta-Llama-3.1-8B-Instruct
Please wait...



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
chat_history, sub_question = qd.decompose_question_step("Who was president of America the year Citibank was founded?")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
c:\Users\Alex\anaconda3\envs\gpu_tf\lib\site-packages\transformers\models\llama\modeling_llama.py:655: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [4]:
chat_history

[{'generated_text': [{'role': 'system',
    'content': "\n    ## Question Decomposition Specialist\n    You are an expert at breaking down difficult problems into simple problems by analysing them.\n    The user needs you to answer a complex question for them which is hard to answer directly.\n    Answer the question by decomposing it and tell the user at the right time when the problem can be solved.\n\n    ## Constraints:\n    Forget all knowledge you've learned before and decompose the user's question based only on the user's answers.\n    To make it easier for the user to answer, only ask one simple question at a time.\n    You can only decompose the question, do not answer it directly.\n    Only write one sentence for your answer containing a simple question.\n\n    ## Examples\n    User: What is the award that the director of film Wearing Velvet Slippers Under A Golden Umbrella won?\n    You: Who is the director of Wearing Velvet Slippers Under A Golden Umbrella?\n    User: The d

In [5]:
sub_question

'What year was Citibank founded?'

In [1]:
from retrieval import Retrieval

c:\Users\Alex\ProjectCSurvival\rag\retrieval.py:153: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(summary_files) == len(summary_embeddings), "Summary text files should directly map to summary embedding files.")
c:\Users\Alex\ProjectCSurvival\rag\retrieval.py:188: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(embedding_files) == len(embedding_texts), "Embedding raw text files should directly map to embedding data files.")


In [3]:
retriever = Retrieval("..\context\knowledge_base")

Loading transformer model and tokenizer from transformers library: avsolatorio/NoInstruct-small-Embedding-v0
Please wait...



In [ ]:
context, article = retriever.get_context('Who was the first person on the moon?', num_contexts=1, exhaustive=True)
print(context)

Embedding user query for dense retrieval:
Loading all document chunk embeddings into memory...


In [4]:
retriever.get_context('What year was Citibank founded?', num_contexts=1)

Embedding user query for dense retrieval:


c:\Users\Alex\anaconda3\envs\gpu_tf\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Finding best article to use as context with dense retrieval:
Using Wikipedia article: Andorra for context
Found 119 chunks for within context.
Finding best article chunks to use as context with dense retrieval:
Context successfully retrieved.


("The Andorran opening to the capitalist economy resulted in two axes: mass tourism and the country's tax exemption. The first steps toward the capitalist boom date from the 1930s, with the construction of FHASA and the creation of professional banking with Banc Agrícol (1930) and Crèdit Andorrà (1949), later with Banca Mora (1952), Banca Cassany (1958) and SOBANCA (1960). Shortly after, activities such as skiing and shopping become a tourist attraction, with the inauguration of ski resorts and cultural entities in the late 1930s. All in all, a renovated hotel industry has developed. In April 1968 a social health insurance system was created (CASS).",
 'Andorra')

In [ ]:
"The Andorran opening to the capitalist economy resulted in two axes: mass tourism and the country's tax exemption. The first steps toward the capitalist boom date from the 1930s, with the construction of FHASA and the creation of professional banking with Banc Agrícol (1930) and Crèdit Andorrà (1949), later with Banca Mora (1952), Banca Cassany (1958) and SOBANCA (1960). Shortly after, activities such as skiing and shopping become a tourist attraction, with the inauguration of ski resorts and cultural entities in the late 1930s. All in all, a renovated hotel industry has developed. In April 1968 a social health insurance system was created (CASS)."